**determine who was**
* best dressed
* worst dressed 
* most controversial

In [112]:
import pandas as pd

df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

data = df['text']

In [113]:
import re
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

In [114]:
import spacy
# python -m spacy download en
nlp = spacy.load('en')
doc = nlp(cleanse(data[0]))
print([(w.text, w.pos_) for w in doc])

[('JLo', 'PROPN'), ("'s", 'PART'), ('dress', 'NOUN'), (' ', 'SPACE'), ('#', 'SYM'), ('eredcarpet', 'NOUN'), ('#', 'SYM'), ('GoldenGlobes', 'PROPN')]


In [115]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [116]:
def remove_apostrophe(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    return text

# test
print(remove_apostrophe("Tina 's"))

Tina


In [117]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [118]:
import pprint
def print_top(n, num):
    sorted_nominees = sorted(n.items(), key=lambda e: e[1], reverse=True)
    pprint.pprint(sorted_nominees[0:num])
#     names = [pair[0] for pair in sorted_nominees]
#     pprint.pprint(names[0:num])
    print('list length:', len(n))

In [119]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [120]:
def remove_apostrophe(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    return text

# test
print(remove_apostrophe("Tina 's"))

Tina


In [121]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    matches = re.findall(pattern, line)
    for match in matches:
        line = re.sub(match, ' ', line)
    return line

In [122]:
from textblob import TextBlob
# import nltk
# nltk.download('punkt')
# pip3 install TextBlob

def get_polarity(line):
    blob = TextBlob(line)
    for sentence in blob.sentences:
#         print(sentence.sentiment.polarity)
        return float(sentence.sentiment.polarity)
    
line = "Jennifer Lopez's dress is jaw droppingly amazing #GoldenGlobes #redcarpet"
get_polarity(line)

0.6000000000000001

In [131]:
%%time
from fuzzywuzzy import fuzz
from textblob import TextBlob

def find_names():
    pattern = re.compile("(dress)|(wear)|(outfit)|(suit)|(cloth)", re.IGNORECASE)
    # pattern1 = re.compile("(\saward\s)|(\sbest\s)", re.IGNORECASE)
    neg_dict = {}
    pos_dict = {}

    for line in data:
        line = cleanse(line)
        line = remove_retweet_prefix(line)
        line = remove_at(line)
        line = remove_hashtag(line)
        line = remove_url(line)
        match = pattern.search(line)

        if match:
            p = get_polarity(line)
            doc = nlp(line)
            
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    name = ent.text.strip()
                    name = remove_apostrophe(name)
                    # not add to list if the entity is null or golden globe related
                    if p is None or name == '' or fuzz.ratio(name.lower(), 'golden globes') > 60:
                        continue
                    if p < 0:
                        if name in neg_dict:
                            neg_dict[name] += p
                        else:
                            neg_dict[name] = p
                    
                    if p > 0:
                        if name in pos_dict:
                            pos_dict[name] += p
                        else:
                            pos_dict[name] = p

#     pos = {key: value for key, value in persons_dic.items() if value > 10}
    # nominees = sorted(nominees.items(), key=lambda e: e[1], reverse=True)
    
    # get rid of names that are too long or too short
    for name in list(pos_dict.keys()):
        if len(name) < 5 or len(name) > 17:
            del pos_dict[name]
            
    for name in list(neg_dict.keys()):
        if len(name) < 5 or len(name) > 17:
            del neg_dict[name]
            
    return pos_dict, neg_dict

pos_dict, neg_dict = find_names()

CPU times: user 2min 52s, sys: 19.6 s, total: 3min 12s
Wall time: 49 s


In [132]:
# find the best dressed
print_top(pos_dict, 10)

# pprint.pprint(pos_dict)

[('Kate Hudson', 87.11540404040403),
 ('Lucy Liu', 40.93333333333334),
 ('Claire Danes', 39.35575396825397),
 ('Jennifer Lawrence', 38.71820226070226),
 ('Jessica Alba', 36.355),
 ('Anne Hathaway', 28.426388888888887),
 ('Jennifer Lopez', 24.532196969696955),
 ('Kerry Washington', 22.09953102453103),
 ('Nicole Kidman', 18.383333333333333),
 ('Jessica Chastain', 16.816849747474745)]
list length: 403


In [133]:
# find the worst dressed
sorted_neg = sorted(neg_dict.items(), key=lambda e: e[1], reverse=False)
pprint.pprint(sorted_neg[0:10])
print(len(sorted_neg))
# pprint.pprint(neg_dict)

[('Sienna Miller', -14.383333333333333),
 ('Kate Hudson', -12.045833333333333),
 ('Alexander McQueen', -11.555555555555548),
 ('Lucy Liu', -10.37375541125541),
 ('Jennifer Lawrence', -9.366666666666667),
 ('Halle Berry', -7.711111111111111),
 ('Anne Hathaway', -4.796153846153847),
 ('Nicole Kidman', -4.666666666666666),
 ('Lena Dunham', -4.366666666666666),
 ('Lucy Lui', -2.6333333333333337)]
145


In [134]:
def get_name_to_reduce(nominees):
    names_clusters = []
    names = list(nominees.keys())

    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)

        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)

        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    #     print(cluster)


    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)
#     print('\nnames clusters to merge:')
#     pprint.pprint(names_clusters_reduced)
#     print('\n')
    return names_clusters_reduced

In [135]:
# merge names using cluster
def reduce_names(nominees):
    reduced_nominees = nominees.copy()
    names_clusters_reduced = get_name_to_reduce(nominees)

    def weighted_freq(element):
        if element in reduced_nominees:
            return abs(reduced_nominees[element]) * len(element)
        else:
            return 0

    for cluster in names_clusters_reduced:
        # select the longest entity name
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in reduced_nominees and selected_entity_name in reduced_nominees:
                reduced_nominees[selected_entity_name] += reduced_nominees[name]
                del reduced_nominees[name]
    return reduced_nominees

In [136]:
# merge similar names, find top 20
reduced_pos = reduce_names(pos_dict)
print_top(reduced_pos, 20)

[('Kate Hudson', 89.4320707070707),
 ('Jennifer Lawrence', 82.83838457838456),
 ('Lucy Liu', 53.15416666666666),
 ('Claire Danes', 42.780753968253975),
 ('Jessica Alba', 39.17999999999999),
 ('Nicole Kidman', 31.183333333333334),
 ('Anne Hathaway', 29.493055555555554),
 ('Julianne Moore', 24.17899711399711),
 ('Kerry Washington', 23.385245310245313),
 ('Jessica Chastain', 18.406849747474745),
 ('Eva Longoria', 13.41984126984127),
 ('Halle Berry', 13.160648148148146),
 ('Mermaid Dresses', 12.972727272727273),
 ('Taylor Swift', 12.831666666666665),
 ('Jodie Foster', 12.647619047619042),
 ('Bradley Cooper', 12.049999999999999),
 ('Miu Miu', 11.566666666666665),
 ('Emily Blunt', 10.802380952380954),
 ('Amy Poehler', 10.731406926406926),
 ('Lea Michele', 10.339444444444444)]
list length: 265


In [137]:
# merge similar names, find top 20
reduced_neg = reduce_names(neg_dict)
sorted_neg = sorted(reduced_neg.items(), key=lambda e: e[1], reverse=False)
pprint.pprint(sorted_neg[0:20])
print(len(sorted_neg))

[('Sienna Miller', -16.53333333333333),
 ('Lucy Liu', -14.807088744588741),
 ('Jennifer Lawrence', -13.76845238095238),
 ('Kate Hudson', -12.045833333333333),
 ('Alexander McQueen', -11.555555555555548),
 ('Halle Berry', -8.719444444444445),
 ('Anne Hathaway', -5.8961538461538465),
 ('Nicole Kidman', -4.666666666666666),
 ('Lena Dunham', -4.366666666666666),
 ("Daniel Day Lewis'", -2.9976190476190476),
 ('Jessica Chastain', -2.6025),
 ('Eva Longoria', -2.5263888888888886),
 ('PHOTOS', -2.45),
 ('Kristen Wiig', -2.386111111111111),
 ('Rosario Dawson', -2.35),
 ('Jodie Foster', -1.6714285714285713),
 ('Terrible Dress', -1.6666666666666667),
 ('Disappointed', -1.5),
 ('Amy Poehler', -1.3769345238095239),
 ('Guiliana', -1.2833333333333332)]
104


In [138]:
# find the most controversial 
def get_sorted_names(nominees, f):
    sorted_nominees = sorted(nominees.items(), key=lambda e: e[1], reverse=f)
    names = [pair[0] for pair in sorted_nominees]
    return names

pos_names = get_sorted_names(reduced_pos, True)
neg_names = get_sorted_names(reduced_neg, False)

contro_dict = {}
for name in pos_names:
    if name in neg_names:
        contro_dict[name] = pos_names.index(name) + neg_names.index(name)

pprint.pprint(contro_dict)

{'Alexander McQueen': 33,
 'Amanda Seyfried': 67,
 'Amy Adams': 107,
 'Amy Poehler': 36,
 'Angelina Jolie': 293,
 'Anne Hathaway': 12,
 'Ben Affleck': 80,
 'Bill Clinton': 67,
 'Claire Danes': 23,
 'Damien Lewis': 105,
 'Debra Messing': 115,
 'Don Cheadle': 179,
 'Elie Saab': 196,
 'Eva Longoria': 21,
 'Halle Berry': 16,
 'Heidi Klum': 209,
 'Helen Hunt': 108,
 'Helen Mirren': 85,
 'Hugh Jackman': 159,
 'Jennifer Lawrence': 3,
 'Jessica Alba': 35,
 'Jessica Chastain': 19,
 'Jodie Foster': 29,
 'Julianne Moore': 61,
 'Kate Hudson': 3,
 'Kelly Osbourne': 68,
 'Kerry Washington': 69,
 'Lena Dunham': 42,
 'Looks': 167,
 'Lucy Liu': 3,
 'Marie Antoinette': 248,
 'Megan Fox': 59,
 'Michael Kors': 98,
 'Naeem Khan': 90,
 'Nicole Kidman': 12,
 'Oscar': 148,
 'PHOTOS': 45,
 'Quentin Tarantino': 202,
 'Rosario Dawson': 46,
 'Sarah Hyland': 157,
 'Sienna Miller': 39,
 'Sofia Vergara': 60,
 'Starbucks': 253,
 'Tattoos': 275,
 'Taylor Swift': 38,
 'Tina Fey': 76}
